In [ ]:
import numpy as np
import pandas as pd 

df_train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
df_example = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

df_train

In [ ]:
df_test

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.info()
df_test.info()

In [ ]:
df_train['target'].value_counts()

In [ ]:
X = df_train.loc[:, df_train.columns != 'target']
X = X.loc[:, X.columns != 'id']
y = df_train.loc[:, df_train.columns == 'target']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
X_norm = scaler.transform(X)
print(X_norm)

In [ ]:
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, accuracy_score

catboost = CatBoostClassifier()
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_norm, y, test_size=0.3)

In [ ]:
catboost.fit(X_train, Y_train)
y_pred = catboost.predict(X_test)
print(y_pred)
print(accuracy_score(Y_test, y_pred))

In [ ]:
y_pred_proba = catboost.predict_proba(X_test)
print(y_pred_proba)

In [ ]:
X_final = df_test.loc[:, df_test.columns != 'id']

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_final)
X_norm_final = scaler.transform(X_final)
print(X_norm_final)

In [ ]:
y_pred_final = catboost.predict(X_norm_final)
print(y_pred_final)
y_pred_proba_final = catboost.predict_proba(X_norm_final)
print(y_pred_proba_final)

In [ ]:
target_pred = []
for i in range(len(y_pred_proba_final)):
    target_pred.append(y_pred_proba_final[i][1])

id_test = []
for i in range (600000, 1139999+1):
    id_test.append(i)

d = {"id": pd.Series(id_test), "target": pd.Series(target_pred)}

output = pd.DataFrame(d)
print(output)

In [ ]:
output.to_csv('Tab-Nov-Mandrov.csv', index=False)
output